In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.prompt { display:none !important; }</style>"))

In [2]:
from ipyleaflet import Map
basic_map = Map(zoom = 1)
basic_map

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [3]:
import gpxpy
gpx_file_path = 'Gran_Fondo_Porto_SantElpidio_2022.gpx'

with open(gpx_file_path) as fh:
    gpxFile = gpxpy.parse(fh)

In [4]:
from statistics import mean
#import ipyleaflet
from ipyleaflet import basemaps, FullScreenControl, LayerGroup, \
    MeasureControl, Polyline, Marker, CircleMarker, WidgetControl
from ipywidgets import Button, HTML, HBox, VBox, Checkbox, FileUpload, \
    Label, Output, IntSlider, Layout, Image, link
from bqplot import pyplot as plt
from bqplot import *

In [5]:
#def plot_map(gpx):
"""
Plot the GPS trace on a map
"""
from ipyleaflet import Map

points = [p.point for p in gpxFile.get_points_data(distance_2d=True)]
mean_lat = mean(p.latitude for p in points)
mean_lng = mean(p.longitude for p in points)

# create the map
m = Map(center=(mean_lat, mean_lng), zoom=12, basemap=basemaps.Stamen.Terrain)

# show trace
line = Polyline(locations=[[[p.latitude, p.longitude] for p in points],],
                color = "red", fill=False)
m.add_layer(line)

# add markers
waypoints = [
    Marker(location=(point.latitude, point.longitude), title=point.name,
           popup=HTML(value=point.name), draggable=False)
    for point in gpxFile.waypoints
]
waypoints_layer = LayerGroup(layers=waypoints)
#m.add_layer(waypoints_layer)

# add a checkbox to show / hide waypoints
waypoints_checkbox = Checkbox(value=True, description='Show Waypoints')

def update_visible(change):
    for p in waypoints:
        p.visible = change['new']

waypoints_checkbox.observe(update_visible, 'value')
waypoint_control = WidgetControl(widget=waypoints_checkbox, position='bottomright')
m.add_control(waypoint_control)

# enable full screen mode
m.add_control(FullScreenControl())

# add measure control
measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
)
m.add_control(measure)

#return m
m

Map(center=[43.14745246278689, 13.682265709631148], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [6]:
# https://www.youtube.com/watch?v=jAKUPm-Cul0
#j = 90
#gpxFile.get_points_data(distance_2d=True)[j]#.tracks().segments[j]

In [7]:
#def plot_elevation(gpx):
"""
Return an elevation graph for the given gpx trace
"""
points = gpxFile.get_points_data(distance_2d=True)
px = [p.distance_from_start / 1000 for p in points]
py = [p.point.elevation for p in points]

x_scale, y_scale = LinearScale(), LinearScale()
x_scale.allow_padding = False
x_ax = Axis(label='Distance (km)', scale=x_scale)
y_ax = Axis(label='Elevation (m)', scale=y_scale, orientation='vertical')

lines = Lines(x=px, y=py, scales={'x': x_scale, 'y': y_scale})

u = 90
elevation = Figure(title='Elevation Chart %d' %u, axes=[x_ax, y_ax], marks=[lines])


elevation.layout.width = 'auto'
elevation.layout.height = 'auto'
elevation.layout.min_height = '300px'

#elevation.interaction = IndexSelector(scale=x_scale)
#return elevation
elevation

Figure(axes=[Axis(label='Distance (km)', scale=LinearScale(allow_padding=False)), Axis(label='Elevation (m)', …

In [25]:
import pandas as pd
j = 0
points = []
for segment in gpxFile.tracks[j].segments:
    for p in segment.points:
        points.append({
            'time': p.time,
            'latitude': p.latitude,
            'longitude': p.longitude,
            'elevation': p.elevation,
        })
coords = pd.DataFrame.from_records(points)
coords.head()

,time,latitude,longitude,elevation
0,2022-04-18 00:00:00+00:00,43.23870,13.75904,19.0
1,2022-04-18 00:00:02+00:00,43.23864,13.75896,19.0
2,2022-04-18 00:00:02+00:00,43.23860,13.75896,20.0
3,2022-04-18 00:00:04+00:00,43.23855,13.75898,19.0
4,2022-04-18 00:00:05+00:00,43.23850,13.75907,19.0
